# Preparation 

In [ ]:
#Download telegram on your phone or/and computer

#Create an bot and copy the token to "botToken" variabele 
##See this link for instructions https://medium.com/@ManHay_Hong/how-to-create-a-telegram-bot-and-send-messages-with-python-4cf314d9fa3e

#To get your 'botChatID' do
##search on telegram @chatid_echo_bot and tap /start. It will echo your chat id.

#Choose the process you want to monitor it uses: ps aux|grep <process>

In [ ]:
import requests
import subprocess
import time

##########################################
#To Config <----------------------------
botToken = 'PLEASE-FILL'
botChatID = 'PLEASE-FILL'
processToCheck='testing.py' #what process should be checked #ps aux|grep <process>

processTestInterval_sleepTime=30 #secounds in between each test run
maxNumSendMsg_failedProcess=5 #Maximum number of messages after process failed

### Helper function

In [ ]:
#Wrap test, so it is better to see the message
def addTestFrame(text_str):
    text_str='#'*10+'\n'+text_str+'\n'+'#'*10
    return str(text_str)

## Function to send a message via telegram

In [ ]:
def telegram_bot_sendMessageV2(message):
    payload = {
        'chat_id': botChatID,
        'text': message,
        'parse_mode': 'HTML'}
    return requests.post("https://api.telegram.org/bot{token}/sendMessage".format(token=botToken), data=payload).content

## Get the state of a process via 'ps aux'

In [ ]:
def getProceessStatus(processStr):
    cmd = "ps aux|grep '"+processStr+"'|grep -v grep |awk '{print $0}'"
    result = subprocess.run(cmd, stdout=subprocess.PIPE,shell=True)
    psaux_result=((result.stdout).decode('utf-8')).split()
    
    #Test whether process is running
    if len(psaux_result)==0:
        return False #Process is not running
    else:
        return str(psaux_result) #Process is running, return process details

## Loop to check every periode the process


In [ ]:
maxSendCounter=0

def testProcessStatusAndSend(notificationIfRunning=False):
    global maxSendCounter
    
    psaux_result=getProceessStatus(processToCheck)
    
    #Decide what notification sending to the telegram bot
    if psaux_result==False and maxSendCounter<=maxNumSendMsg_failedProcess:
        message=str('!!!!!!!!!!! Process: '+ processToCheck+ ' is not running anymore !!!!!!!!!!!')
        maxSendCounter+=1
        return telegram_bot_sendMessageV2(message)
    
    #Send a message that the process is still running
    if psaux_result!=False and notificationIfRunning==True:
        message='Process: '+ processToCheck+' is still running'
        maxSendCounter=0 #Set counter back to zero afer process is back running
        return telegram_bot_sendMessageV2(message)
    
    #Send message if the process is running again after the process stopped
    if psaux_result!=False and maxSendCounter>0:
        message='Process: '+ processToCheck+' is again !!!!!!!!!!!'
        maxSendCounter=0 #Set counter back to zero afer process is back running
        return telegram_bot_sendMessageV2(message)    

## 'Main' function
#### This function will loop and checks the process

In [ ]:
startTime=time.time() #Start time
hoursRunning=1 #Send still alive after one hour
#numSendMsg_failedProcess_couner=0 #Used for counting
#Send start notification
telegram_bot_sendMessageV2(addTestFrame('Start watching process: '+processToCheck))
testProcessStatusAndSend(notificationIfRunning=True)
while True:
    #Check process
    
    testProcessStatusAndSend()
    
    #Wait until the next check
    time.sleep(processTestInterval_sleepTime)
    
    #Sends every hour a still alive message
    if time.time()>(hoursRunning*60*60)+startTime:
        #Send hourly still alive message
        message=':) Still watching process: '+ processToCheck+'!!!'
        telegram_bot_sendMessageV2(message)
        hoursRunning+=1